In [ ]:
import os
import pickle
import numpy as np
import detectron2
import torch
import mrcnn_utils
import mrcnn_visualize
import pathlib
import pandas as pd
import skimage
import skimage.io

import seaborn as sns
import skimage.measure

import evaluate
import analyze
import matplotlib.pyplot as plt

In [ ]:
dataset_name_ = 'satellite'

gt_root = pathlib.Path('../data/raw/via_2.0.8/via_satellite_masks.json')
pred_root = pathlib.Path('../data/interim/satellite_predictions_outputs_np.pickle')

assert gt_root.is_file()
assert pred_root.is_file()

In [ ]:
with open(pred_root, 'rb') as f:
    pred_data = pickle.load(f)

pred_instances = [analyze.instance_set().read_from_numpy_out(
    x,y,True) for x, y in pred_data.items()]
    
    
gt_instances = [analyze.instance_set().read_from_ddict(x, True)
             for x in analyze.get_data_dicts(gt_root)]

gt_instances = [{x.filepath.name: x for x in gt_instances}[f] 
                for f in pred_data] # hacky but this rearranges 
                # the order of the instances so both sets are the same
                # the dictionary maps filenames to the gt instances
                # f in pred_data  iterates through ordered filenames
                # stored as keys in the dictionary

In [ ]:
# ground truth
gt = gt_instances[-1]

# predicted
pred = pred_instances[-1]


In [ ]:
# ground truth
colors = evaluate.random_colors(len(gt.class_idx), 32)
mrcnn_visualize.display_instances(gt.img, gt.boxes, gt.masks, 
                                  gt.class_idx, [''], colors=colors)

In [ ]:
## original mask predictions 

pcolors = evaluate.random_colors(len(pred.class_idx), 34)
mrcnn_visualize.display_instances(gt.img, pred.boxes, pred.masks, 
                                  pred.class_idx, [''], colors=pcolors)

In [ ]:
results = {}
for predset, label in zip([pred_instances,],
                         ['pred',]):
    results[label] = []
    for gt, pred in zip(gt_instances, predset):
        results[label].append(analyze.mask_match_stats(gt.masks, 
                                               pred.masks))
        

In [ ]:
match_precision = [x['match_precision'] for x in results['pred']]
match_recall = [x['match_recall'] for x in results['pred']]

mask_precision = [x['mask_precision'] for x in results['pred']]
mask_recall = [x['mask_recall'] for x in results['pred']]


In [ ]:
files = ['Train {}'.format(x) for x in range(1, 9)]
files.extend(['Validation {}'.format(x) for x in range(1, 3)]) 
files_list = []
[files_list.extend(files) for _ in range(2)]


y = np.concatenate((match_precision, 
                    match_recall,), axis=0)

x = []
[x.extend([i]*10) for i in ['precision', 'recall', ]]

df_match = pd.DataFrame()

df_match['files'] = files_list
df_match['y'] = y
df_match['x'] = x

fig, ax = plt.subplots(figsize=(6,3), dpi=300)
sns.barplot(x="x", y="y",  hue='files', data=df_match, ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel('match precision or recall')
ax.set_xlabel('')
xlims = ax.get_xlim()
ax.plot(xlims, [1,1], '--k', linewidth=1)
ax.set_xlim(xlims)
ax.set_ylim([0.5,1.01])
plt.show()


In [ ]:
y = np.concatenate((mask_precision, mask_recall, ), axis=0)

newx = []
newy = []
newfiles = []

for xi, yi, file in zip(x, y, files_list):
    for yii in yi:
        newx.append(xi)
        newy.append(yii)
        newfiles.append(file)
df_mask = pd.DataFrame()

df_mask['files'] = newfiles
df_mask['y'] = newy
df_mask['x'] = newx

fig, ax = plt.subplots(figsize=(6,3), dpi=300)
#box = ax.get_position()
#ax.set_position([0,0,box.width,box.height])
sns.boxplot(x="x", y="y",  hue='files', data=df_mask, 
            ax=ax, showfliers=False)
l1 = ax.legend(bbox_to_anchor=(1.0,1.0))
ax.set_ylabel('mask precision or recall')
ax.set_xlabel('')
xlims = ax.get_xlim()
ax.plot(xlims, [1,1], '--k', linewidth=1)
ax.set_xlim(xlims)
plt.show()

In [ ]:
pred = pred_instances[-1]
gt = gt_instances[-1]
masks, boxes, colors, colormap = analyze.match_visualizer(gt.masks, gt.boxes,
                                  pred.masks, pred.boxes,)

mrcnn_visualize.display_instances(gt.img, boxes, masks, 
                                  np.zeros(len(boxes), np.int), [''], colors=colors)
plt.show()

fig, ax = plt.subplots(1,1)
for i, name in enumerate(['TP','FP','FN']):
    ax.bar(-1000-i, 4+i, color=colormap[name], label=name)
    
ax.axis([0,10,0,10])
ax.legend(fontsize=16)
ax.set_title('legend for image plot')
    
plt.show()


In [ ]:
masks, colormap = analyze.mask_visualizer(gt.masks, pred.masks)
bbox = np.asarray([(0,0,*masks.shape[:2]) for _ in range(masks.shape[2])])

img = gt.img.copy()
colors = sns.color_palette(n_colors=7)
colors[1] = [1,0,0]
colors[2] = [0,1,0]
colors[3] = [0.8, 0.8, 0.8]

for i, (mask, color) in enumerate(zip(np.transpose(masks, (2,0,1)), colors)):
    img = mrcnn_visualize.apply_mask(img, mask, color)

fig, ax = plt.subplots(1,1, figsize=(15,10), dpi=300)
plt.imshow(img)


fig, ax = plt.subplots(1,1)
for i in range(7):
    ax.bar(-1000-i, 4+i, color=colors[i], label=colormap[1][i+1])
    
ax.axis([0,10,0,10])
ax.legend(fontsize=16)
ax.set_title('legend for image plot')
    
plt.show()


In [ ]:
save_data=False
if save_data:
    for iset, name in zip([gt_instances, pred_instances], ['gt', 'pred']):
        with open('../data/interim/instance_sets/{}_{}_instance_sets.pickle'.format(dataset_name, name), 'wb') as f:
            pickle.dump(iset, f)